In [3]:
import pandas as pd

df = pd.read_csv('governor_deployments.csv')
df.head()

,platform,platform_id,name,website,additional,proposals_count
0,governor,00233765-eb17-4e75-954f-82258bb990aa,Shell Protocol,NaN,NaN,0
1,governor,0032e64d-5105-4369-89c4-46f05247c684,Free Company,NaN,NaN,0
2,governor,00453db4-685a-4221-9ad1-fda0d82c3a8a,Cream Finance,NaN,NaN,46
3,governor,004556b2-f0c0-4843-887c-4b539140ee43,Bleeps DAO,NaN,NaN,0
4,governor,0050d8cf-2650-4d66-94dc-77d871b0d755,Psychedelic,NaN,NaN,0


In [4]:
# if votes count column exists, quit
if 'votes_count' in df.columns:
    print('votes_count column already exists')
    raise SystemExit(99)

In [5]:
votes_df = pd.read_csv('../../votes/governor/governor_votes.csv')
votes_df.head()

,platform,platform_deployment_id,proposal_id,vote_id,voter,date,choice,weight
0,governor,e1a6f432-bc3c-438e-a07b-ce35095de79a,0xec568fffba86c094cf06b22134b23074dfe2252c-110,0x7ae109a63ff4dc852e063a673b40bed85d22e585-110,0x7ae109a63ff4dc852e063a673b40bed85d22e585,2022-10-22 06:31:59,FOR,9.500005e+22
1,governor,e1a6f432-bc3c-438e-a07b-ce35095de79a,0xec568fffba86c094cf06b22134b23074dfe2252c-110,0x62a43123fe71f9764f26554b3f5017627996816a-110,0x62a43123fe71f9764f26554b3f5017627996816a,2022-10-21 19:32:35,FOR,8.600291e+22
2,governor,e1a6f432-bc3c-438e-a07b-ce35095de79a,0xec568fffba86c094cf06b22134b23074dfe2252c-110,0x212baf65228fe03674da0efb88cac7eadb276db5-110,0x212baf65228fe03674da0efb88cac7eadb276db5,2022-10-20 02:39:35,FOR,1.000000e+22
3,governor,e1a6f432-bc3c-438e-a07b-ce35095de79a,0xec568fffba86c094cf06b22134b23074dfe2252c-110,0x5b3bffc0bcf8d4caec873fdcf719f60725767c98-110,0x5b3bffc0bcf8d4caec873fdcf719f60725767c98,2022-10-20 18:50:35,FOR,3.323895e+21
4,governor,e1a6f432-bc3c-438e-a07b-ce35095de79a,0xec568fffba86c094cf06b22134b23074dfe2252c-110,0xb6d844e2529bc79a444662aa55f72aca08fd8f54-110,0xb6d844e2529bc79a444662aa55f72aca08fd8f54,2022-10-20 09:35:59,FOR,1.250194e+20


In [6]:
votes_count_df = votes_df.groupby('platform_deployment_id')['vote_id'].count()
votes_count_df = votes_count_df.reset_index()
# rename vote_id to votes_count
votes_count_df = votes_count_df.rename(columns={'vote_id': 'votes_count'})
votes_count_df.head()

,platform_deployment_id,votes_count
0,1a11c997-d9c2-4d3c-abe4-3d85190d02f2,216
1,32b7bb94-fbc9-4172-9219-0ed422edf0ce,42
2,33c90aeb-e7eb-4656-8663-f4831dc63468,57
3,4e758c1e-8410-4fa4-9924-6bead3afde57,296
4,4fa685ba-26e9-40cb-b98c-cb9182a3b407,27025


In [7]:
# merge
df = pd.merge(df, votes_count_df, left_on='platform_id', right_on='platform_deployment_id', how='left', validate='1:1')
df.head()

,platform,platform_id,name,website,additional,proposals_count,platform_deployment_id,votes_count
0,governor,00233765-eb17-4e75-954f-82258bb990aa,Shell Protocol,NaN,NaN,0,NaN,NaN
1,governor,0032e64d-5105-4369-89c4-46f05247c684,Free Company,NaN,NaN,0,NaN,NaN
2,governor,00453db4-685a-4221-9ad1-fda0d82c3a8a,Cream Finance,NaN,NaN,46,NaN,NaN
3,governor,004556b2-f0c0-4843-887c-4b539140ee43,Bleeps DAO,NaN,NaN,0,NaN,NaN
4,governor,0050d8cf-2650-4d66-94dc-77d871b0d755,Psychedelic,NaN,NaN,0,NaN,NaN


In [8]:
# count rows with votes
df.shape[0] - df['votes_count'].isna().sum()

21

In [9]:
df.votes_count.describe()

count        21.000000
mean      19653.047619
std       75765.388165
min          18.000000
25%          59.000000
50%         296.000000
75%        2964.000000
max      349082.000000
Name: votes_count, dtype: float64

In [10]:
# fill nans with 0
df['votes_count'] = df['votes_count'].fillna(0)
# make sure it's an int
df['votes_count'] = df['votes_count'].astype(int)
# drop platform_deployment_id
df = df.drop(columns=['platform_deployment_id'])
df = df[['platform', 'platform_id', 'name', 'website', 'additional', 'votes_count', 'proposals_count']]
df.head()

,platform,platform_id,name,website,additional,votes_count,proposals_count
0,governor,00233765-eb17-4e75-954f-82258bb990aa,Shell Protocol,NaN,NaN,0,0
1,governor,0032e64d-5105-4369-89c4-46f05247c684,Free Company,NaN,NaN,0,0
2,governor,00453db4-685a-4221-9ad1-fda0d82c3a8a,Cream Finance,NaN,NaN,0,46
3,governor,004556b2-f0c0-4843-887c-4b539140ee43,Bleeps DAO,NaN,NaN,0,0
4,governor,0050d8cf-2650-4d66-94dc-77d871b0d755,Psychedelic,NaN,NaN,0,0


In [11]:
# save
df.to_csv('governor_deployments.csv', index=False)